### Analyse de Subway_in  :
**Code Bokeh qui fonctionne pas bien. Il faudrait faire un checkbox en menu déroulant, pas tout afficher d'un coup, un bouton pour tout désactiver car 40 plot c'est long. Rajouter un 'hover', etc...**

In [11]:
import torch
from torch.cuda.amp import autocast, GradScaler

# Initialisation de GradScaler pour la gestion de la précision mixte
scaler = GradScaler()

# Créer un tenseur sparse
indices = torch.tensor([[0, 1], [2, 0]], device='cuda')  # Indices de la matrice sparse sur GPU
values = torch.tensor([3.0, 4.0], dtype=torch.float32, device='cuda')  # Valeurs sur GPU
size = torch.Size([3, 3])
support = torch.sparse.FloatTensor(indices.t(), values, size).to('cuda')  # Matrice sparse sur GPU

# Créer un tenseur dense en float16 pour mixed precision
x0 = torch.rand((3, 2), dtype=torch.float16, device='cuda')  # Exemple de matrice dense sur GPU

# Utiliser autocast pour la précision mixte
with autocast():
    # Convertir x0 en float32 pour l'opération sparse
    x0_float32 = x0.to(torch.float32)  # Convertir x0 en float32

    # Multiplication avec la matrice sparse
    support_float32 = support.to(torch.float32)  # Assurez-vous que support est float32
    x1 = torch.sparse.mm(support_float32, x0_float32)  # Multiplication en float32

    # Continuer avec d'autres opérations en précisions mixtes
    # result = some_other_operation(x1)  # Exemples d'autres opérations

# Utilisation de GradScaler pour la rétropropagation
# optimizer.zero_grad()
# scaler.scale(loss).backward()  # Exemple de calcul de perte
# scaler.step(optimizer)
# scaler.update()

print("Result:", x1)

RuntimeError: "addmm_sparse_cuda" not implemented for 'Half'

In [9]:
import torch
device = 'cuda'

t = torch.randn(3,3).to(torch.float16).to(device).to_sparse_csr()
a = torch.randn(3,3).to(torch.float16).to(device).to_sparse_csr()
torch.sparse.mm(t,a)

tensor(crow_indices=tensor([0, 3, 6, 9]),
       col_indices=tensor([0, 1, 2, 0, 1, 2, 0, 1, 2]),
       values=tensor([ 0.7266, -0.0333, -0.1309, -2.1055,  1.4854,  0.1091,
                       3.5449, -1.5596, -0.5527]), device='cuda:0',
       size=(3, 3), nnz=9, dtype=torch.float16, layout=torch.sparse_csr)

In [23]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [3]:
import numpy as np 

from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import Legend, LegendItem,HoverTool
from bokeh.palettes import Turbo256
from bokeh.layouts import row

# Personnal Import 
from utils.utilities_DL import load_raw_data
from dataset import DataSet
# ...

# Paths
FOLDER_PATH = 'data/'

#Init and load data: 
H,W,D = 6,1,1
L = H+W+D 
step_ahead = 1

window_pred = np.arange(2*96)

FILE_NAME = 'subway_IN_interpol_neg_15_min_2019_2020.csv'



In [4]:
# Load df, remove forbidden values: 
df,invalid_dates,time_step_per_hour = load_raw_data(FOLDER_PATH,FILE_NAME,single_station = False)
dataset = DataSet(df,time_step_per_hour=time_step_per_hour, Weeks = W, Days = D, historical_len= H,step_ahead=step_ahead)
dataset.get_shift_between_set()   # get shift indice and shift date from the first element / between each dataset 
dataset.get_feature_vect()  # Build 'df_shifted'.
dataset.remove_forbidden_prediction(invalid_dates) # Build 'df_verif' , which is df_shifted without sequences which contains invalid date

# Find atypical Value. Check if we need to modify them.
valid_df = dataset.df[~(dataset.df.index.isin(invalid_dates))]
limit = 1500
all_plot = False

p = figure(x_axis_type='datetime', height=600, width=1400, sizing_mode="scale_width")

# Stocker les renderers pour les ajouter à la légende
renderers = []

if all_plot:
    for i, column in enumerate(df.columns):
        x, y = df.index, df[column]
        color = Turbo256[int(256 / len(df.columns) * i)]  
        renderer = p.line(x, y, line_width=2, color=color, alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)
        renderer.visible = False
        renderers.append(renderer)

    legend = Legend(items=[LegendItem(label=col, renderers=[rend], index=i) for i, (col, rend) in enumerate(zip(df.columns, renderers))])

else:

    for i,column in enumerate(valid_df):
        color = Turbo256[int(256 / len(df.columns) * i)]  
        sub_df = valid_df[valid_df[column] > limit]
        if not sub_df.empty:
            x,y = sub_df.index,sub_df[column]
            renderer = p.scatter(x, y, color=color, alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)
            renderers.append(renderer)

    x,y = dataset.df.index,dataset.df.quantile(0.9,axis = 1)
    renderer = p.line(x, y, line_width=2, color='black', alpha=0.8, legend_label='all_stations', muted_color=color, muted_alpha=0)
    renderers.append(renderer)

    legend = Legend(items=[LegendItem(label=col, renderers=[rend], index=i) for i, (col, rend) in enumerate(zip(list(df.columns)+['all_stations'], renderers))])  


p.add_layout(legend, 'right')
p.legend.click_policy = "mute"
hover = HoverTool()
p.add_tools(hover)
output_notebook()
show(p) 

saving = False
if saving:
    output_file("df_IN_outliers.html")
save(p)

TypeError: load_raw_data() missing 1 required positional argument: 'FILE_NAME'

In [ ]:
p = figure(x_axis_type='datetime', height=600, width=1400, sizing_mode="scale_width")

column = 'Ampère Victor Hugo'
column = 'Bellecour'
x,y = dataset.df.index,dataset.df[column]

renderer = p.line(x, y, color='blue', alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)

output_notebook() 
show(p)

output_file(f"df_IN_{column}.html")
save(p)
